## Write by Chengzhi Yuan @April 29 2023

### Some advanced models: 

###### 一些闲着没事儿或者不想学习了写的模型和简单的特征工程，可能有些效果不好，当个模板用

including:

- Sparse feature split via chi2select,

- XGB, Cascade SVC, Stacking XGB SVC MLP,

- Wide and Deep with transpose transformers and MoE,

- MoE with transpose transformer expert,

- and ablation study, fm, etc.

- further ......

In [1]:
import pandas as pd
import pdb
from sklearn import preprocessing
import numpy as np
from sklearn.feature_selection import chi2, SelectKBest

import sys
sys.path.append('.')

import importlib

import xgboost
from xgboost import XGBClassifier

from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression

# pdb.set_trace()

train_data = pd.read_csv('./train.csv')
test_data = pd.read_csv('./test.csv')
# pdb.set_trace()


import warnings

warnings.filterwarnings('ignore')

pass

### Data Processing

In [2]:
sparse_features_name = ['HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'VIP', 'Name']
dense_features_name  = list(set(train_data.columns.tolist()[:-1]) - set(['PassengerId']) - set(sparse_features_name))
label_data = train_data['Transported']

labels = []
for _ in range(label_data.shape[0]):
    if label_data[_] == True:
        labels.append(1)
    else:
        labels.append(0)
labels = np.array(labels)

# pdb.set_trace()

all_features_name = sparse_features_name + dense_features_name
# label_encoder = preprocessing.LabelEncoder()
# all_features_list = []
# for feature_name in all_features_name:
#     train_data[feature_name] = train_data[feature_name].fillna(method = 'pad')
#     all_features_list.append(label_encoder.fit_transform(train_data[feature_name]))
# all_features = np.array(all_features_list).T


# all_features = np.zeros((all_features.shape[0], len(all_features_name)))
# scaler = preprocessing.MinMaxScaler()
# for idx, feature_name in enumerate(dense_features_name):
#     tmp = train_data[feature_name].fillna(train_data[feature_name].mean())
#     dense_features[:, idx] = scaler.fit_transform(np.array(tmp).reshape(-1, 1)).reshape(-1)


label_encoder = preprocessing.LabelEncoder()
sparse_features_list = []
for feature_name in sparse_features_name:
    train_data[feature_name] = train_data[feature_name].fillna(method = 'pad')
    sparse_features_list.append(label_encoder.fit_transform(train_data[feature_name]))
sparse_features = np.array(sparse_features_list).T

# pdb.set_trace()


dense_features = np.zeros((sparse_features.shape[0], len(dense_features_name)))
scaler = preprocessing.MinMaxScaler()
for idx, feature_name in enumerate(dense_features_name):
    tmp = train_data[feature_name].fillna(train_data[feature_name].mean())
    # dense_features[:, idx] = scaler.fit_transform(np.array(tmp).reshape(-1, 1)).reshape(-1)
    dense_features[:, idx] = np.array(tmp)    
    # pdb.set_trace()

def analysis():
    tmp = train_data[dense_features_name].values.astype(np.float32)
    nanlocations = np.argwhere(np.isnan(tmp))
    nanattributesindex = nanlocations[:, -1].tolist()

    ret = [nanattributesindex.count(_) for _ in range(len(dense_features_name))]
    pdb.set_trace()

# analysis()
# pdb.set_trace()
# dense_features = np.array(dense_features_list)

'''
# chi2 evaluation
chi2_model = SelectKBest(chi2, k = len(train_data.columns.tolist()[:-1]) - 1)
print ('=================chi2==================')

# pdb.set_trace()

chi2_model.fit_transform(np.concatenate((sparse_features, dense_features), axis = -1), labels)
print (chi2_model.scores_)
print (chi2_model.pvalues_)
print (sparse_features_name + dense_features_name)
'''



################## tabular data augmentation ####################
pass

### FIXME: test data process

In [3]:
sparse_features_testlist = []
# label_encoder.fit(sparse_features_name)
for feature_name in sparse_features_name:
    test_data[feature_name] = test_data[feature_name].fillna(method = 'pad')
    sparse_features_testlist.append(label_encoder.fit_transform(test_data[feature_name]))
    test_sparse_features = np.array(sparse_features_testlist).T

test_dense_features = np.zeros((test_sparse_features.shape[0], len(dense_features_name)))
scaler = preprocessing.MinMaxScaler()
for idx, feature_name in enumerate(dense_features_name):
    tmp = test_data[feature_name].fillna(test_data[feature_name].mean())
    test_dense_features[:, idx] = scaler.fit_transform(np.array(tmp).reshape(-1, 1)).reshape(-1)

### Model Training and Evaluation

In [4]:
# all_features = np.concatenate((sparse_features, dense_features), axis = -1)
# all_features = sparse_features
all_dense_features = dense_features

# pdb.set_trace()

assert all_dense_features.shape[0] == labels.shape[0]

# use_fm = 0

# CryoSleep classifier
print ('\n')
print ('=================split based "CyroSleep"==================')

all_input_sparsefeatures = np.concatenate((sparse_features[:, 0].reshape(-1, 1), sparse_features[:, 2:].reshape(-1, 4)), axis = -1)
# all_input_sparsefeatures = np.concatenate((sparse_features[:, 0].reshape(-1, 1), sparse_features[:, 2:].reshape(-1, 4)), axis = -1)
# all_input_sparsefeatures = sparse_features.reshape(-1, 6)



# pdb.set_trace()


### Ablation Study: FM Feature Cross
use_fm = 0
if use_fm:
    fm = lambda tmp: 0.5 * (np.power(np.sum(tmp, axis = 1), 2) - np.sum(np.power(tmp, 2), axis = 1))
    fm_feature = fm(np.expand_dims(all_input_sparsefeatures, axis = -1))
    assert fm_feature.shape[0] == all_input_sparsefeatures.shape[0]

    all_dense_features = np.concatenate((all_dense_features, fm_feature), axis = -1)
    assert all_dense_features.shape[-1] == dense_features.shape[-1] + 1

assert all_dense_features.shape[0] == labels.shape[0]

all_densefeatures_model1 = all_dense_features[train_data[train_data['CryoSleep'] == True].index.tolist()]
all_sparsefeatures_model1 = all_input_sparsefeatures[train_data[train_data['CryoSleep'] == True].index.tolist()]
all_label_model1    = labels[train_data[train_data['CryoSleep'] == True].index.tolist()]

# pdb.set_trace()

all_densefeatures_model2 = all_dense_features[train_data[train_data['CryoSleep'] == False].index.tolist()]
all_sparsefeatures_model2 = all_input_sparsefeatures[train_data[train_data['CryoSleep'] == False].index.tolist()]
all_label_model2    = labels[train_data[train_data['CryoSleep'] == False].index.tolist()]

# pdb.set_trace()

# assert (all_features_model1.shape[0] + all_features_model2.shape[0] == train_data.shape[0])



=================split based "CyroSleep"==================


### Naive XGB

In [5]:

print ('\n')
print ('=================train xgb model=====================')
xgb = XGBClassifier(learning_rate = 0.01, n_estimators = 20, max_depth = 8, \
                     min_child_weight = 1, gamma = 0., subsample = 1, \
                     colsample_btree = 1, scale_pos_weight = 1, random_state = 100, slient = 0)

xgb.fit(all_dense_features[:-1000], labels[:-1000])

## FIXME: predict
y_pred = xgb.predict(all_dense_features[-1000:, :])
y_eval = labels[-1000:]

acc_count = 0
for _ in range(y_pred.shape[0]):
    if y_pred[_] == y_eval[_]: acc_count += 1
print ('model accuarte rate: ' + str(acc_count / y_eval.shape[0]))

# pdb.set_trace()

xgb1 = XGBClassifier(learning_rate = 0.01, n_estimators = 20, max_depth = 8, \
                     min_child_weight = 1, gamma = 0., subsample = 1, \
                     colsample_btree = 1, scale_pos_weight = 1, random_state = 100, slient = 0)

xgb1.fit(np.concatenate((all_densefeatures_model1[:-1000], all_sparsefeatures_model1[:-1000]), axis = -1), \
         all_label_model1[:-1000])

pred_xgb1 = xgb1.predict(np.concatenate((all_densefeatures_model1[-1000:], all_sparsefeatures_model1[-1000:]), axis = -1))
cal_acc = lambda pred, labels: np.sum(pred == labels) / pred.shape[0]
print ('accuracy 1: ' + str(cal_acc(pred_xgb1, all_label_model1[-1000:])))



xgb2 = XGBClassifier(learning_rate = 0.01, n_estimators = 20, max_depth = 8, \
                     min_child_weight = 1, gamma = 0., subsample = 1, \
                     colsample_btree = 1, scale_pos_weight = 1, random_state = 100, slient = 0)


xgb2.fit(np.concatenate((all_densefeatures_model2[:-1000], all_sparsefeatures_model2[:-1000]), axis = -1), \
         all_label_model2[:-1000])
pred_xgb2 = xgb2.predict(np.concatenate((all_densefeatures_model2[-1000:], all_sparsefeatures_model2[-1000:]), axis = -1))
print ('accuracy 2: ' + str(cal_acc(pred_xgb2, all_label_model2[-1000:])))



# pdb.set_trace()
        




=================train xgb model=====================
[14:29:12] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-3.7/xgboost/src/learner.cc:627: 
Parameters: { "colsample_btree", "slient" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


model accuarte rate: 0.779
[14:29:12] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-3.7/xgboost/src/learner.cc:627: 
Parameters: { "colsample_btree", "slient" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any s

### Cascade SVC

In [6]:
from sklearn.multioutput import ClassifierChain

base_svc = SVC()

chains1 = [ClassifierChain(base_svc, order = "random", random_state = i) for i in range(10)]
for chain1 in chains1:
    # pdb.set_trace()
    chain1.fit(np.concatenate((all_densefeatures_model1[:-1000], all_sparsefeatures_model1[:-1000]), axis = -1),\
              all_label_model1[:-1000].reshape(-1, 1))

model1_pred_chains = np.array([chain1.predict(np.concatenate((all_densefeatures_model1[-1000:], all_sparsefeatures_model1[-1000:]), axis = -1)) \
                               for chain1 in chains1])

def get_ret(tmp: np.array) -> np.array:
    ret = np.zeros((tmp.shape[0], 1))
    for _ in range(tmp.shape[0]):
        ret[_, :] = np.argmax(np.bincount(tmp[_, :].reshape(-1).astype(np.int64)))
    # pdb.set_trace()
    return ret

pred_cascadesvc1 = get_ret(model1_pred_chains.transpose(1, 0, 2)).reshape(-1)
cal_acc = lambda pred, labels: np.sum(pred == labels) / pred.shape[0]

# pdb.set_trace()

print ('model1 accuracy: ' + str(cal_acc(pred_cascadesvc1, all_label_model1[-1000:])))





chains2 = [ClassifierChain(base_svc, order = "random", random_state = i) for i in range(10)]
for chain2 in chains2:
    # pdb.set_trace()
    chain2.fit(np.concatenate((all_densefeatures_model2[:-1000], all_sparsefeatures_model2[:-1000]), axis = -1),\
              all_label_model2[:-1000].reshape(-1, 1))

model2_pred_chains = np.array([chain2.predict(np.concatenate((all_densefeatures_model2[-1000:], all_sparsefeatures_model2[-1000:]), axis = -1)) \
                               for chain2 in chains2])

pred_cascadesvc2 = get_ret(model2_pred_chains.transpose(1, 0, 2)).reshape(-1)

print ('model2 accuracy: ' + str(cal_acc(pred_cascadesvc2, all_label_model2[-1000:])))





model1 accuracy: 0.821
model2 accuracy: 0.781


### Wide&Deep (with transpose transformer and MoE)

In [7]:
import torch
import torch.nn as nn
import torch.utils.data as Data
import torch.nn.functional as F
from tqdm import trange
import math
import mixture_of_experts as moe

# pdb.set_trace()

train_set_1 = Data.TensorDataset(torch.FloatTensor(all_densefeatures_model1[:-1000]), \
                                 torch.FloatTensor(all_sparsefeatures_model1[:-1000]), \
                                 torch.FloatTensor(all_label_model1[:-1000].reshape(-1, 1)))
eval_set_1 = Data.TensorDataset(torch.FloatTensor(all_densefeatures_model1[-1000:]), \
                                torch.FloatTensor(all_sparsefeatures_model1[-1000:]), \
                                torch.FloatTensor(all_label_model1[-1000:].reshape(-1, 1)))

train_set_2 = Data.TensorDataset(torch.FloatTensor(all_densefeatures_model2[:-1000]), \
                                 torch.FloatTensor(all_sparsefeatures_model2[:-1000]), \
                                 torch.FloatTensor(all_label_model2[:-1000]).reshape(-1, 1))
eval_set_2 = Data.TensorDataset(torch.FloatTensor(all_densefeatures_model2[-1000:]), \
                                torch.FloatTensor(all_sparsefeatures_model2[-1000:]), \
                                torch.FloatTensor(all_label_model2[-1000:]).reshape(-1, 1))


train_set_3 = Data.TensorDataset(torch.FloatTensor(all_dense_features[:-2000]), \
                                 torch.FloatTensor(all_input_sparsefeatures[:-2000]), \
                                 torch.LongTensor(labels[:-2000]).reshape(-1, 1))
eval_set_3 = Data.TensorDataset(torch.FloatTensor(all_dense_features[-2000:]), \
                                torch.FloatTensor(all_input_sparsefeatures[-2000:]), \
                                torch.LongTensor(labels[-2000:]).reshape(-1, 1))
# pdb.set_trace()

train_loader_1 = Data.DataLoader(train_set_1, batch_size = 32, shuffle = True)
train_loader_2 = Data.DataLoader(train_set_2, batch_size = 32, shuffle = True)
train_loader_3 = Data.DataLoader(train_set_3, batch_size = 32, shuffle = True)

eval_loader_1 = Data.DataLoader(eval_set_1, batch_size = 32, shuffle = False)
eval_loader_2 = Data.DataLoader(eval_set_2, batch_size = 32, shuffle = False)
eval_loader_3 = Data.DataLoader(eval_set_3, batch_size = 32, shuffle = False)



class PositionalEncoding(nn.Module):

    def __init__(self, d_model: int, dropout: float = 0.5, max_len: int = 5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Args:
            x: Tensor, shape [seq_len, batch_size, embedding_dim]
        """
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)
        

######################################################################################    
'''

Transposed transformers with the naive MoE

'''


class model(nn.Module):
    def __init__(self):
        super().__init__()
        self.emb_dense = nn.Linear(1, 256)
        self.emb_sparse = nn.Sequential(nn.Linear(5, 256), nn.BatchNorm1d(256), nn.ReLU(),
                                        nn.Linear(256, 64), nn.BatchNorm1d(64), nn.ReLU(),
                                        nn.Linear(64, 32))
        
        self.pos_encoder = PositionalEncoding(d_model = 256, dropout = 0.5)
        self.transformer_encoder_layer = nn.TransformerEncoderLayer(d_model = 256, nhead = 8)
        self.transformer_encoder = nn.TransformerEncoder(self.transformer_encoder_layer, num_layers = 3)
        self.transformer_encoder2 = nn.TransformerEncoder(self.transformer_encoder_layer, num_layers = 3)
        
        self.moe_model = moe.MoE(dim = 256, num_experts = 8, hidden_dim = 256 * 4, activation = nn.ReLU, 
                                 second_policy_train = 'random', second_policy_eval = 'random',
                                 second_threshold_train = 0.2, second_threshold_eval = 0.2,
                                 capacity_factor_train = 1.25, capacity_factor_eval = 2., loss_coef = 1e-2)
        
        self.mlp_head = nn.Sequential(nn.Linear(6 * 256, 32), nn.BatchNorm1d(32), nn.ReLU())
        self.fc = nn.Sequential(nn.Linear(16, 1), nn.Sigmoid())
        
    def forward(self, x: torch.FloatTensor, sparse_features: torch.FloatTensor) -> torch.FloatTensor:  # 
        '''
        x: dense_feature: [batch_size, 6]
        sparse_features: [batch_size, 5]
        '''
        batch_size, feature_num = x.size()
        # pdb.set_trace()
        x = self.pos_encoder(self.emb_dense(x.unsqueeze(dim = -1)))
        out1 = self.transformer_encoder(x) # out1 shape [batch, feature_num, 256]

        # pdb.set_trace()
        '''
        MoE model
        pass
        import mixture_of_experts as moe
        '''
        
        out_intermedia, aux_loss = self.moe_model(out1)  # out ret shape == input shape
        
        out2 = self.transformer_encoder2(out_intermedia.permute(1, 0, 2)).permute(1, 0, 2) 
        
        # pdb.set_trace()
        
        out3 = F.glu(self.mlp_head(out2.reshape(batch_size, feature_num * 256)))
        
        sparse_embedding = F.glu(self.emb_sparse(sparse_features))
        
        final_out = self.fc(out3 + sparse_embedding)
        # pdb.set_trace()
        return final_out, aux_loss

    
class train_eval_session():
    
    def __init__(self):
        pass
        
    @staticmethod
    def run(model: nn.Module, train_loader_x: Data.DataLoader, eval_loader_x: Data.DataLoader):
        # torch.compile(model) # for torch 2.0 and above
        
        criterion = nn.BCELoss()
        device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
        
        optimizer = torch.optim.Adam(model.parameters(), lr = 1e-3)
        model.train()
        n_epochs = 5
        for current_epoch in trange(n_epochs):
            for _, batch_data in enumerate(train_loader_x):
                train_features, train_sparse_features, labels = batch_data[0].to(device), batch_data[1].to(device), batch_data[2].to(device)
            
                # pdb.set_trace()
            
                pred, aux_loss = model(train_features, train_sparse_features)
                # pdb.set_trace()
                loss = criterion(pred, labels.type(torch.float)) + aux_loss
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                # print (loss.cpu().item())
            print ('current epoch: ' + str(current_epoch) + ' | ' + 'loss: ' + str(loss.cpu().item()))
        
        with torch.no_grad():
            pass
            model.eval()
        
            all_eval_pred = []
            all_eval_labels = []
            for _, eval_data in enumerate(eval_loader_x):
                eval_features, eval_sparse_features, eval_labels = eval_data[0].to(device), eval_data[1].to(device), eval_data[2].to(device)
            
                pred_out, _ = model(eval_features, eval_sparse_features) # eval features shape [batch_size, feature_num]  
                # pdb.set_trace()

                all_eval_pred  += (pred_out > 0.5).long().view(-1).tolist()   # for sigmoid
                # all_eval_pred   += torch.max(pred_out, dim = -1)[1].view(-1).tolist()  # for softmax
                all_eval_labels += eval_labels.view(-1).tolist()
        
            # pdb.set_trace()
        
            acc = np.sum(np.array(all_eval_pred) == np.array(all_eval_labels)) / np.array(all_eval_pred).shape[0]
            print ('accuracy: ' + str(acc))        


model1 = model(); model2 = model(); model3 = model()

print ('----------------model1----------------')
train_eval_session.run(model1, train_loader_1, eval_loader_1)

print ('----------------model2----------------')
train_eval_session.run(model2, train_loader_2, eval_loader_2)

print ('----------------model3----------------')
train_eval_session.run(model3, train_loader_3, eval_loader_3)

----------------model1----------------


 20%|█████████████████████▌                                                                                      | 1/5 [00:23<01:35, 23.83s/it]

current epoch: 0 | loss: 0.6826862096786499


 40%|███████████████████████████████████████████▏                                                                | 2/5 [00:47<01:10, 23.51s/it]

current epoch: 1 | loss: 0.5035728812217712


...

 80%|██████████████████████████████████████████████████████████████████████████████████████▍                     | 4/5 [04:58<01:14, 74.55s/it]

current epoch: 3 | loss: 0.7906532287597656


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [06:13<00:00, 74.76s/it]

current epoch: 4 | loss: 0.7560297250747681


accuracy: 0.564


In [ ]:
######################################################################################    
'''

MoE with the transposed transformer

'''
class tranpose_transformer_expert(nn.Module):
    def __init__(self):
        super().__init__()
        self.transformer_encoder_layer = nn.TransformerEncoderLayer(d_model = 256, nhead = 8)
        self.transformer_encoder = nn.TransformerEncoder(self.transformer_encoder_layer, num_layers = 3)
        self.transformer_encoder2 = nn.TransformerEncoder(self.transformer_encoder_layer, num_layers = 3)
    
    def forward(self, x: torch.FloatTensor) -> torch.FloatTensor:
        '''
        x: (num_of_experts, batch_size, dim)
        '''
        out1 = self.transformer_encoder(x)
        out2 = self.transformer_encoder2(out1.permute(1, 0, 2)).permute(1, 0, 2)
        return out2

    
class moe_tt_model(nn.Module):

    def __init__(self):
        super().__init__()
        self.dense_emb = nn.Linear(1, 256)
        self.sparse_emb_1 = nn.Sequential(nn.Linear(5, 256), nn.ReLU(),
                                       nn.Linear(256, 6))
        self.sparse_emb_2 = nn.Linear(1, 256)
        self.MoE_transpose_transformer = moe.MoE(dim = 256, num_experts = 8, hidden_dim = 256 * 4, activation = nn.ReLU, 
                                     second_policy_train = 'random', second_policy_eval = 'random',
                                     second_threshold_train = 0.2, second_threshold_eval = 0.2,
                                     capacity_factor_train = 1.25, capacity_factor_eval = 2., loss_coef = 1e-2)
        self.head = nn.Sequential(nn.Linear(256 * 6, 1), nn.Sigmoid())
    
    def forward(self, x: torch.FloatTensor, sparse_features: torch.FloatTensor) -> torch.FloatTensor:
        '''
        x: shape (batch_size, dense_feature_num)
        sparse_features: shape (batch_size, sparse_feature_num)
        ''' 
        dense_embedding = self.dense_emb(x.unsqueeze(dim = -1)) # [batch_size, 6, 256]
        # pdb.set_trace()
        sparse_embedding = self.sparse_emb_2(self.sparse_emb_1(sparse_features).unsqueeze(dim = -1)) # [batch_size, 6, 256]
        out1, aux_loss = self.MoE_transpose_transformer(dense_embedding + sparse_embedding) # same as input
        return self.head(out1.reshape(x.shape[0], -1)), aux_loss

    

moe_tt_model1 = moe_tt_model()
moe_tt_model2 = moe_tt_model()
moe_tt_model3 = moe_tt_model()


print ('----------------model1----------------')
train_eval_session.run(moe_tt_model1, train_loader_1, eval_loader_1)

print ('----------------model2----------------')
train_eval_session.run(moe_tt_model2, train_loader_2, eval_loader_2)

print ('----------------model3----------------')
train_eval_session.run(moe_tt_model3, train_loader_3, eval_loader_3)


### Stacking Model (xgb, mlp, svc)

In [ ]:
# pdb.set_trace()

############# model: mlp + xgb + svc stacking method ##################
print ('\n')
print ('=================train ensemble model1=====================')
xgb1 = XGBClassifier(learning_rate = 0.01, n_estimators = 20, max_depth = 8, \
                     min_child_weight = 1, gamma = 0., subsample = 1, \
                     colsample_btree = 1, scale_pos_weight = 1, random_state = 100, slient = 0)
mlp1 = MLPClassifier()
svc1 = SVC()

xgb1.fit(np.concatenate((all_densefeatures_model1[:-1000], all_sparsefeatures_model1[:-1000]), axis = -1), \
         all_label_model1[:-1000])
mlp1.fit(np.concatenate((all_densefeatures_model1[:-1000], all_sparsefeatures_model1[:-1000]), axis = -1), \
         all_label_model1[:-1000])
svc1.fit(np.concatenate((all_densefeatures_model1[:-1000], all_sparsefeatures_model1[:-1000]), axis = -1), \
         all_label_model1[:-1000])

stacking_model_1 = StackingClassifier(
    estimators = [('xgb', xgb1), ('mlp', mlp1), ('svc', svc1)],
    final_estimator = LogisticRegression())

stacking_model_1.fit(np.concatenate((all_densefeatures_model1[:-1000], all_sparsefeatures_model1[:-1000]), axis = -1), \
                     all_label_model1[:-1000])


## FIXME: predict
y_pred = stacking_model_1.predict(np.concatenate((all_densefeatures_model1[-1000:], all_sparsefeatures_model1[-1000:]), axis = -1))
y_eval = all_label_model1[-1000:]

acc_count = 0
for _ in range(y_pred.shape[0]):
    if y_pred[_] == y_eval[_]: acc_count += 1

# pdb.set_trace()
        
print ('model1 accuarte rate: ' + str(acc_count / y_eval.shape[0]))



print ('\n')
print ('=================train ensemble model2=====================')
xgb2 = XGBClassifier(learning_rate = 0.01, n_estimators = 20, max_depth = 8, \
                     min_child_weight = 1, gamma = 0., subsample = 1, \
                     colsample_btree = 1, scale_pos_weight = 1, random_state = 100, slient = 0)
mlp2 = MLPClassifier()
svc2 = SVC()

xgb2.fit(np.concatenate((all_densefeatures_model2[:-1000], all_sparsefeatures_model2[:-1000]), axis = -1), \
         all_label_model2[:-1000])
mlp2.fit(np.concatenate((all_densefeatures_model2[:-1000], all_sparsefeatures_model2[:-1000]), axis = -1), \
         all_label_model2[:-1000])
svc2.fit(np.concatenate((all_densefeatures_model2[:-1000], all_sparsefeatures_model2[:-1000]), axis = -1), \
         all_label_model2[:-1000])

stacking_model_2 = StackingClassifier(
    estimators = [('xgb', xgb2), ('mlp', mlp2), ('svc', svc2)],
    final_estimator = LogisticRegression())

stacking_model_2.fit(np.concatenate((all_densefeatures_model2[:-1000], all_sparsefeatures_model2[:-1000]), axis = -1), \
                     all_label_model2[:-1000])

### FIXME: predict
y_pred_2 = stacking_model_2.predict(np.concatenate((all_densefeatures_model2[-1000:], all_sparsefeatures_model2[-1000:]), axis = -1))
y_eval_2 = all_label_model2[-1000:]



acc_count_2 = 0
for _ in range(y_pred_2.shape[0]):
    if y_pred_2[_] == y_eval_2[_]: acc_count_2 += 1

print ('model2 accurate rate: ' + str(acc_count_2 / y_eval_2.shape[0]))

### Ablation Study: Without Feature Split

In [ ]:
# pdb.set_trace()

print ('\n')
print ('=================train ensemble model3=====================')
xgb1 = XGBClassifier(learning_rate = 0.01, n_estimators = 20, max_depth = 8, \
                     min_child_weight = 1, gamma = 0., subsample = 1, \
                     colsample_btree = 1, scale_pos_weight = 1, random_state = 100, slient = 0)
mlp1 = MLPClassifier()
svc1 = SVC()

# pdb.set_trace()

all_train_features = np.concatenate((all_features_model1[:-1000], all_features_model2[:-1000]), axis = 0)
all_train_labels = np.concatenate((all_label_model1[:-1000], all_label_model2[:-1000]), axis = 0)

xgb1.fit(all_train_features, all_train_labels)
mlp1.fit(all_train_features, all_train_labels)
svc1.fit(all_train_features, all_train_labels)

stacking_model_3 = StackingClassifier(
    estimators = [('xgb', xgb1), ('mlp', mlp1), ('svc', svc1)],
    final_estimator = LogisticRegression())

stacking_model_3.fit(all_train_features, all_train_labels)

def cal_overall_accuracy(all_input_features, all_ground_truth_label_s):
    all_y_pred = stacking_model_3.predict(all_input_features)
    
    # acc = 0
    # pdb.set_trace()
    
    
    assert all_y_pred.shape[0] == all_ground_truth_label_s.shape[0]
    
    return np.sum(all_y_pred == all_ground_truth_label_s) / all_y_pred.shape[0]
    
print ('Model results without feature split: ')    
print (cal_overall_accuracy(all_features_model1[-1000:], all_label_model1[-1000:]))
print (cal_overall_accuracy(all_features_model2[-1000:], all_label_model2[-1000:]))

############# AUTOML: (autogluon ?) ####################

pass

### Next Ablation Study: Fourier Transform (Fourier Analysis) & Floor divide & etc.

In [ ]:
pass

### test

In [ ]:
################### test #######################

# pdb.set_trace()
ret = []
for _ in range(test_data.shape[0]):
    if test_data['CryoSleep'][_] == True:
        tmp_ret = xgb1.predict(np.concatenate((test_sparse_features[_, 0].reshape(-1, 1), \
                               test_sparse_features[_, 2:].reshape(1, -1), \
                               test_dense_features[_, :].reshape(1, -1)), axis = -1))

    else:
        tmp_ret = xgb2.predict(np.concatenate((test_sparse_features[_, 0].reshape(1, -1), \
                               test_sparse_features[_, 2:].reshape(1, -1), \
                               test_dense_features[_, :].reshape(1, -1)), axis = -1))
    if tmp_ret == 1: ret.append(True)
    else: ret.append(False)


ret_df = {'PassengerId': test_data['PassengerId'], 'Transported': ret}
ret_pd_df = pd.DataFrame(ret_df)
ret_pd_df.to_csv('./result.csv', index = False)